# Part 5 - Explore the Clusters
In this notebook, I will explore the clusters created by KMeans and create an interactive Folium map of the clusters.  

<div style="text-align: left">
    <h2>
        <a href="https://nbviewer.jupyter.org/github/KathrynDH/IBMCapstoneFinalProject/blob/master/Cluster%20Metro%20Areas.ipynb">&larr; Part 4</a>
    </h2>
</div>


In [19]:
#import libraries
import requests
import numpy as np
import pandas as pd
import ibm_boto3
import io

In [20]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [21]:
from IPython.display import Image 
from IPython.core.display import HTML 
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl

In [22]:
# The code was removed by Watson Studio for sharing.

In [23]:
#Import dataframe
obj = resource.Object(bucket_name=bucket, key='VenueDataWithClusters.json').get()
df = pd.read_json(io.BytesIO(obj['Body'].read()))
df.sort_index(inplace=True)
df.head()

,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
0,Tokyo,Japan,Asia,35.682839,139.759455,Café,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Lounge,Garden,French Restaurant,Dessert Shop,Thai Restaurant,Steakhouse,2
1,Delhi,India,Asia,28.651718,77.221939,Indian Restaurant,Bar,Café,Flea Market,Lounge,Asian Restaurant,Ice Cream Shop,South Indian Restaurant,Portuguese Restaurant,Plaza,3
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café,1
3,São Paulo,Brazil,South America,-23.550651,-46.633382,Brazilian Restaurant,Japanese Restaurant,Café,Bakery,Snack Place,Bookstore,Cosmetics Shop,Asian Restaurant,Art Gallery,Chinese Restaurant,1
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Bar,Café,Shopping Mall,Pizza Place,Hotel,Art Gallery,Italian Restaurant,Market,1


In [24]:
#How many metro areas are in each cluster?
df_count = df.loc[:,['Metropolitan','Cluster Labels']].groupby('Cluster Labels').count()
df_count

,Metropolitan
Cluster Labels,
0,1
1,19
2,18
3,8
4,17
5,2
6,1
7,3
8,1


In [25]:
#Look at the metro areas in each cluster
for i in df_count.index:
    print('Cluster ',i)
    with pd.option_context('display.max_rows', None):
        display(df[df['Cluster Labels']==i])

Cluster  0


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
17,Istanbul,Turkey,Europe,41.009633,28.965165,Turkish Restaurant,Jewelry Store,Kebab Restaurant,Café,Restaurant,Garden,Art Gallery,Hotel,Seafood Restaurant,Coffee Shop,0


Cluster  1


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café,1
3,São Paulo,Brazil,South America,-23.550651,-46.633382,Brazilian Restaurant,Japanese Restaurant,Café,Bakery,Snack Place,Bookstore,Cosmetics Shop,Asian Restaurant,Art Gallery,Chinese Restaurant,1
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Bar,Café,Shopping Mall,Pizza Place,Hotel,Art Gallery,Italian Restaurant,Market,1
25,Buenos Aires,Argentina,South America,-34.607562,-58.437076,Argentinian Restaurant,Gym,Bakery,Pizza Place,Ice Cream Shop,Restaurant,Café,Burger Joint,Bar,Plaza,1
27,Paris,France,Europe,48.856610,2.351499,French Restaurant,Ice Cream Shop,Art Gallery,Wine Bar,Bookstore,Burger Joint,Park,Plaza,Coffee Shop,Theater,1
31,Rio de Janeiro,Brazil,South America,-22.911014,-43.209373,Brazilian Restaurant,Bar,Restaurant,Bakery,Gym / Fitness Center,Gym,Café,Coffee Shop,Samba School,Dive Bar,1
35,Rhine-Ruhr,Germany,Europe,51.437302,7.334511,Furniture / Home Store,Bakery,German Restaurant,Café,Ice Cream Shop,Supermarket,Big Box Store,Doner Restaurant,Arts & Crafts Store,Restaurant,1
40,Lima,Peru,South America,-12.062107,-77.036526,Park,Seafood Restaurant,Bar,Shopping Mall,Nightclub,Chinese Restaurant,Peruvian Restaurant,Plaza,Electronics Store,Pub,1
48,Chūkyō (Nagoya),Japan,Asia,35.138700,136.966392,Ramen Restaurant,Japanese Restaurant,Park,Café,Bakery,BBQ Joint,Yoshoku Restaurant,Sake Bar,Chinese Restaurant,Restaurant,1
52,Randstad,Netherlands,Europe,52.378042,5.226072,Sushi Restaurant,Restaurant,Supermarket,Gym,Gym / Fitness Center,Shopping Mall,Grocery Store,Park,Pet Store,Electronics Store,1


Cluster  2


...

,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
26,Xi'an,China,Asia,35.000074,104.999927,Business Service,Women's Store,None,None,None,None,None,None,None,None,13


Cluster  14


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
23,Jakarta,Indonesia,Asia,-6.175394,106.827183,Park,Indonesian Restaurant,Asian Restaurant,Noodle House,Track,Soup Place,Soccer Field,Skate Park,Plaza,Performing Arts Venue,14
50,Bandung,Indonesia,Asia,-6.934469,107.604954,Museum,Noodle House,Park,Pet Store,Restaurant,None,None,None,None,None,14


Cluster  15


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
10,Chengdu,China,Asia,30.662420,104.063322,Chinese Restaurant,Noodle House,Tea Room,Department Store,Szechuan Restaurant,Hotpot Restaurant,Bookstore,Coffee Shop,Shopping Mall,Pedestrian Plaza,15
13,Chongqing,China,Asia,29.558571,106.549282,Shopping Mall,Coffee Shop,Chinese Restaurant,Electronics Store,Park,Nightclub,Asian Restaurant,Museum,Scenic Lookout,Public Art,15
22,Hangzhou,China,Asia,30.248963,120.205234,Chinese Restaurant,Shopping Mall,Movie Theater,Coffee Shop,Plaza,Asian Restaurant,Steakhouse,Sporting Goods Shop,Snack Place,Pedestrian Plaza,15
29,Changzhou,China,Asia,31.812986,119.969205,American Restaurant,Chinese Restaurant,Department Store,Electronics Store,History Museum,Italian Restaurant,Shopping Mall,None,None,None,15
33,Nanjing,China,Asia,32.060974,118.791646,Shopping Mall,Chinese Restaurant,Nightclub,Café,Indian Restaurant,Japanese Restaurant,Multiplex,Electronics Store,Park,Jiangsu Restaurant,15
37,Jinan,China,Asia,36.650701,117.114004,Park,Shopping Mall,Coffee Shop,Department Store,Electronics Store,Fast Food Restaurant,Gourmet Shop,Hotel Bar,Italian Restaurant,Movie Theater,15
45,Guangzhou,China,Asia,23.130196,113.259294,Chinese Restaurant,Nightclub,Shopping Mall,History Museum,Asian Restaurant,Park,Movie Theater,Cantonese Restaurant,Dessert Shop,Clothing Store,15
63,Nanchang,China,Asia,28.679270,115.888599,Grocery Store,History Museum,Lake,Movie Theater,Pub,Shopping Mall,None,None,None,None,15
81,Kanpur,India,Asia,26.460914,80.321759,Shopping Mall,Multiplex,Fast Food Restaurant,Bakery,Clothing Store,Dessert Shop,Diner,Garden Center,Market,Movie Theater,15
89,Shenzhen,China,Asia,22.544570,114.054535,Shopping Mall,Café,Chinese Restaurant,Nightclub,Japanese Restaurant,Lounge,Italian Restaurant,Turkish Restaurant,Thai Restaurant,Park,15


In [26]:
#Create an interactive folium map with different color markers for each cluster
map_clusters = folium.Map(location=[0,0], zoom_start=2)

#set color scheme for the clusters
map_colors = [colors.rgb2hex(cm.tab20b(i)) for i in range(15)]

# add markers to the map
markers_colors = []
for latitude, longitude, metro, country, cluster in zip(df['Latitude'], df['Longitude'], df['Metropolitan'], df['Country'], df['Cluster Labels']):
    label = folium.Popup(str(metro) + ', ' + str(country)+'- Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color=map_colors[cluster-1],
        fill=True,
        fill_color=map_colors[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save('map.html')
map_clusters

### View the interactive map
https://kathryndh.github.io/clustermap.html

In [27]:
#Save the map as an html file
obj = io.open('map.html','rb')
resource.Bucket(name=bucket).put_object(Key='map.html', Body=obj.read())
obj.close()
print('Saved')

Saved


<div style="text-align: left">
    <h2>
        <a href="https://nbviewer.jupyter.org/github/KathrynDH/IBMCapstoneFinalProject/blob/master/Cluster%20Metro%20Areas.ipynb">&larr; Part 4</a>
    </h2>
</div>